In [ ]:
# This program uses machine learning to come up with a simple trading/investing strategy

# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

# Set plot style and filter warnings
plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')

# Load the data
from google.colab import files
uploaded = files.upload()

# Read the CSV file
df = pd.read_csv('aapl.csv')
df

# Set the date column as the index
df.index = pd.to_datetime(df['Date'])
df = df.drop('Date', axis=1)
df

# Create independent variables
df['High_minus_Low'] = df['High'] - df['Low']
df['Open_minus_Close'] = df['Open'] - df['Close']

# Store independent variables in X
X = df[['Open_minus_Close', 'High_minus_Low', 'Close']]
X.head()

# Store target variable in y
y = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)
y

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
df['Predictions'] = model.predict(X)

# Create the trading strategy
df['Position'] = df['Predictions'].diff()

# Calculate returns
df['Returns'] = df['Close'].pct_change()
df['Strategy_Returns'] = df['Position'].shift(1) * df['Returns']

# Calculate cumulative returns
df['Cumulative_Returns'] = (1 + df['Returns']).cumprod()
df['Cumulative_Strategy_Returns'] = (1 + df['Strategy_Returns']).cumprod()

# Plot the results
plt.figure(figsize=(12,6))
plt.plot(df.index, df['Cumulative_Returns'], label='Buy and Hold')
plt.plot(df.index, df['Cumulative_Strategy_Returns'], label='Strategy')
plt.title('Cumulative Returns: Buy and Hold vs Strategy')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()

# Calculate and print the strategy performance
strategy_return = df['Cumulative_Strategy_Returns'].iloc[-1]
buy_hold_return = df['Cumulative_Returns'].iloc[-1]

print(f"Strategy Return: {strategy_return:.2f}")
print(f"Buy and Hold Return: {buy_hold_return:.2f}")